# Cognica를 이용한 IMDB 데이터 필드 검색하기

필드 검색은 데이터 필드의 탐색하여 필터링하는 간단하지만 명확한 검색 방법입니다. 필드 검색을 통해 데이터를 선별하거나 탐색할 수 있습니다. 이 예제는 영화에 대한 정보를 포함하는 IMDB 데이터를 활용합니다.

## 준비 사항

### 의존성 설치

이 예제는 Cognica가 설치되어 있고 서버가 동작하고 있음을 가정합니다. 설치 방법은 [Cognica 설치하기](https://docs.cognica.io/install)를 참고하여 설치할 수 있습니다.

동작에 필요한 라이브러리를 설치합니다.

In [1]:
!pip install -U pip
!pip install cognica

### 데이터 준비하기

데이터는 IMDB 데이터를 사용하며 이 파일(`imdb_top1000.csv`)은 예제에 미리 준비되어 있습니다. 이 데이터는 [IMDB Movies Dataset](https://www.kaggle.com/datasets/harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows)에서 데이터 대한 내용을 확인할 수 있습니다.

데이터를 입수하면 늘 그렇듯 데이터를 살펴보고 의도에 맞게 가공해야 합니다. 데이터의 칼럼은 하나 이상의 타입을 가질 수 있고 이는 데이터를 검색하고 조회하는데 방해가 될 수 있습니다. 특히 대부분 데이터를 다루는 라이브러리에서 NA(Not a Number)는 float 타입이고 이는 불편을 유발하는 경우가 많습니다. 따라서 향후 검색을 편리하도록 하기 위해 몇몇 필드는 타입이 일치하도록 가공합니다. 아래 코드에서는 `read_csv`로 데이터를 로딩하는 과정에서 `converter`를 통해 변환하고 있습니다.

Cognica의 특성상 각 칼럼이 같은 타입을 가지도록 강제하지 않습니다. 이 내용은 [Cognica가 데이터를 다루는 특징](https://docs.cognica.io/user-guide/documentdb/data#cognica%EA%B0%80-%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%A5%BC-%EB%8B%A4%EB%A3%A8%EB%8A%94-%ED%8A%B9%EC%A7%95)에서 참고할 수 있습니다.

In [2]:
import pandas as pd

converters = {
    "Gross": lambda x: int(x.replace(",", "")) if x else 0,
    "Certificate": lambda x: x if x else "",
    "Released_Year": lambda x: 0 if x == "PG" else int(x),
}
df = pd.read_csv("imdb_top_1000.csv", converters=converters)
display(df.dtypes)
df.head()

Poster_Link       object
Series_Title      object
Released_Year      int64
Certificate       object
Runtime           object
Genre             object
IMDB_Rating      float64
Overview          object
Meta_score       float64
Director          object
Star1             object
Star2             object
Star3             object
Star4             object
No_of_Votes        int64
Gross              int64
dtype: object

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,28341469
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,134966411
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,534858444
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,57300000
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,4360000


이제 데이터가 정상적으로 준비되었고 이 데이터를 Cognica에 입력합니다.

## Cognica에 데이터 입력

이제 `create_collection`을 통해 `imdb1`이라는 컬렉션을 생성합니다. 인자에는 `indexes`라는 색인에 대한 정보가 필요하고 여기서는 필수 항목인 `index`를 primary key(`kPrimaryKey`)로 정의하고 나머지 필드는 색인 생성을 하지 않습니다. 검색 성능을 위해서는 색인 정의가 권장됩니다.

그리고 `insert` 함수를 통해 pd.dataframe에서 json으로 변환된 데이터를 입력합니다.

In [3]:
from cognica import Channel, DocumentDB

channel = Channel("localhost", 10080)
doc_db = DocumentDB(channel)

indexes = [{"fields": ["index"], "unique": True, "index_type": "kPrimaryKey"}]

if "imdb1" in doc_db.list_collections():
    doc_db.drop_collection("imdb1")
doc_db.create_collection("imdb1", indexes=indexes)
data = df.reset_index().to_dict(orient="records")
display(data[0])
doc_db.insert("imdb1", data)

{'index': 0,
 'Poster_Link': 'https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_UX67_CR0,0,67,98_AL_.jpg',
 'Series_Title': 'The Shawshank Redemption',
 'Released_Year': 1994,
 'Certificate': 'A',
 'Runtime': '142 min',
 'Genre': 'Drama',
 'IMDB_Rating': 9.3,
 'Overview': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.',
 'Meta_score': 80.0,
 'Director': 'Frank Darabont',
 'Star1': 'Tim Robbins',
 'Star2': 'Morgan Freeman',
 'Star3': 'Bob Gunton',
 'Star4': 'William Sadler',
 'No_of_Votes': 2343110,
 'Gross': 28341469}

## 검색하기

이제 검색할 준비가 되었습니다. `IMDB_Rating`이 8.0 이상인 데이터를 검색합니다. 

In [4]:
doc_db.find("imdb1", {"IMDB_Rating": {"$ge": 8.0}}).head(5)

,Certificate,Director,Genre,Gross,IMDB_Rating,Meta_score,No_of_Votes,Overview,Poster_Link,Released_Year,Runtime,Series_Title,Star1,Star2,Star3,Star4,index
0,A,Frank Darabont,Drama,28341469,9.3,80.0,2343110,Two imprisoned men bond over a number of years...,https://m.media-amazon.com/images/M/MV5BMDFkYT...,1994,142 min,The Shawshank Redemption,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,0
1,A,Francis Ford Coppola,"Crime, Drama",134966411,9.2,100.0,1620367,An organized crime dynasty's aging patriarch t...,https://m.media-amazon.com/images/M/MV5BM2MyNj...,1972,175 min,The Godfather,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1
2,UA,Christopher Nolan,"Action, Crime, Drama",534858444,9.0,84.0,2303232,When the menace known as the Joker wreaks havo...,https://m.media-amazon.com/images/M/MV5BMTMxNT...,2008,152 min,The Dark Knight,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2
3,A,Francis Ford Coppola,"Crime, Drama",57300000,9.0,90.0,1129952,The early life and career of Vito Corleone in ...,https://m.media-amazon.com/images/M/MV5BMWMwMG...,1974,202 min,The Godfather: Part II,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,3
4,U,Sidney Lumet,"Crime, Drama",4360000,9.0,96.0,689845,A jury holdout attempts to prevent a miscarria...,https://m.media-amazon.com/images/M/MV5BMWU4N2...,1957,96 min,12 Angry Men,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,4


다음과 같이 `IMDB_Rating`이 8.0 이상이면서 `Meta_score`가 90 이상인 데이터를 검색할 수 있습니다.

In [5]:
doc_db.find("imdb1", {"IMDB_Rating": {"$ge": 8.0}, "Meta_score": {"$ge": 90}}).head(5)

,Certificate,Director,Genre,Gross,IMDB_Rating,Meta_score,No_of_Votes,Overview,Poster_Link,Released_Year,Runtime,Series_Title,Star1,Star2,Star3,Star4,index
0,A,Francis Ford Coppola,"Crime, Drama",134966411,9.2,100.0,1620367,An organized crime dynasty's aging patriarch t...,https://m.media-amazon.com/images/M/MV5BM2MyNj...,1972,175 min,The Godfather,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1
1,A,Francis Ford Coppola,"Crime, Drama",57300000,9.0,90.0,1129952,The early life and career of Vito Corleone in ...,https://m.media-amazon.com/images/M/MV5BMWMwMG...,1974,202 min,The Godfather: Part II,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,3
2,U,Sidney Lumet,"Crime, Drama",4360000,9.0,96.0,689845,A jury holdout attempts to prevent a miscarria...,https://m.media-amazon.com/images/M/MV5BMWU4N2...,1957,96 min,12 Angry Men,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,4
3,U,Peter Jackson,"Action, Adventure, Drama",377845905,8.9,94.0,1642758,Gandalf and Aragorn lead the World of Men agai...,https://m.media-amazon.com/images/M/MV5BNzA5ZD...,2003,201 min,The Lord of the Rings: The Return of the King,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,5
4,A,Quentin Tarantino,"Crime, Drama",107928762,8.9,94.0,1826188,"The lives of two mob hitmen, a boxer, a gangst...",https://m.media-amazon.com/images/M/MV5BNGNhMD...,1994,154 min,Pulp Fiction,John Travolta,Uma Thurman,Samuel L. Jackson,Bruce Willis,6


아래와 같이 감독이 `Martin Scorsese`인 영화를 검색할 수 있습니다. 여기서 한가지 아쉬운 점은 검색을 위해서 감독의 이름을 정확히 알아야 한다는 것 입니다. 이를 해소하기 위해서 [`$match`](https://docs.cognica.io/user-guide/documentdb/search/operator#%EB%B6%80%EB%B6%84-%EC%9D%BC%EC%B9%98-match)나 [`$regex`](https://docs.cognica.io/user-guide/documentdb/search/operator#%EC%A0%95%EA%B7%9C%EC%8B%9D-%EC%9D%BC%EC%B9%98-regex)를 활용하거나 [FTS](2_fts.ko.ipynb), [Vector 검색](3_vector_search.ko.ipynb)을 이용할 수 있습니다.

In [6]:
doc_db.find("imdb1", {"Director": "Martin Scorsese"}).head(5)

,Certificate,Director,Genre,Gross,IMDB_Rating,Meta_score,No_of_Votes,Overview,Poster_Link,Released_Year,Runtime,Series_Title,Star1,Star2,Star3,Star4,index
0,A,Martin Scorsese,"Biography, Crime, Drama",46836394,8.7,90.0,1020727,The story of Henry Hill and his life in the mo...,https://m.media-amazon.com/images/M/MV5BY2NkZj...,1990,146 min,Goodfellas,Robert De Niro,Ray Liotta,Joe Pesci,Lorraine Bracco,15
1,A,Martin Scorsese,"Crime, Drama, Thriller",132384315,8.5,85.0,1189773,An undercover cop and a mole in the police att...,https://m.media-amazon.com/images/M/MV5BMTI1MT...,2006,151 min,The Departed,Leonardo DiCaprio,Matt Damon,Jack Nicholson,Mark Wahlberg,37
2,A,Martin Scorsese,"Crime, Drama",28262574,8.3,94.0,724636,A mentally unstable veteran works as a nightti...,https://m.media-amazon.com/images/M/MV5BM2M1Mm...,1976,114 min,Taxi Driver,Robert De Niro,Jodie Foster,Cybill Shepherd,Albert Brooks,111
3,A,Martin Scorsese,"Mystery, Thriller",128012934,8.2,63.0,1129894,"In 1954, a U.S. Marshal investigates the disap...",https://m.media-amazon.com/images/M/MV5BYzhiND...,2010,138 min,Shutter Island,Leonardo DiCaprio,Emily Mortimer,Mark Ruffalo,Ben Kingsley,145
4,A,Martin Scorsese,"Biography, Crime, Drama",116900694,8.2,75.0,1187498,"Based on the true story of Jordan Belfort, fro...",https://m.media-amazon.com/images/M/MV5BMjIxMj...,2013,180 min,The Wolf of Wall Street,Leonardo DiCaprio,Jonah Hill,Margot Robbie,Matthew McConaughey,147
